
# MLflow Pydantic AI Flavor

[mlflow.pydantic_ai](https://mlflow.org/docs/latest/api_reference/python_api/mlflow.pydantic_ai.html)

In [0]:
%pip install -U mlflow>=3.8.0
dbutils.library.restartPython()

In [0]:
import mlflow

print(f"MLflow version: {mlflow.__version__}")

## Pydantic AI

GenAI Agent Framework, the Pydantic way

**Pydantic AI** is a Python agent framework designed to help you quickly, confidently, and painlessly build production grade applications and workflows with Generative AI.

[pydantic/pydantic-ai](https://github.com/pydantic/pydantic-ai)


If you know which model you're going to use and want to avoid installing superfluous packages, you can use the [pydantic-ai-slim](https://ai.pydantic.dev/install/#slim-install) package only.

`mcp` is required for `mlflow.pydantic_ai.autolog()` to work:

> mlflow.pydantic_ai: Error importing pydantic_ai.mcp.MCPServer: Please install the `mcp` package to use the MCP server, you can use the `mcp` optional group — `pip install "pydantic-ai-slim[mcp]"`

In [0]:
%pip install -U "pydantic-ai-slim[anthropic,mcp]"
dbutils.library.restartPython()

In [0]:
import pydantic_ai

print(f"Pydantic AI version: {pydantic_ai.__version__}")

# Pydantic AI Autologging

[Tracing PydanticAI](https://docs.databricks.com/aws/en/mlflow3/genai/tracing/integrations/pydantic-ai)

In [0]:
import mlflow
mlflow.pydantic_ai.autolog()

# 👨‍💻 Demo: Hello World

[Hello World Example](https://github.com/pydantic/pydantic-ai?tab=readme-ov-file#hello-world-example)

A minimal example of Pydantic AI


## Create Databricks Secret

[Tutorial: Create and use a Databricks secret](https://docs.databricks.com/aws/en/security/secrets/example-secret-workflow)

<br>

```
databricks secrets create-scope anthropic
```

<br>

```
databricks secrets put-secret anthropic api_key
```

## ANTHROPIC_API_KEY and AnthropicModel

Pydantic AI's [official docs](https://ai.pydantic.dev/models/anthropic/)

https://console.anthropic.com/settings/keys

In [0]:
from pydantic_ai.models.anthropic import AnthropicModel
from pydantic_ai.providers.anthropic import AnthropicProvider


ANTHROPIC_API_KEY = dbutils.secrets.get("anthropic", "api_key")

model = AnthropicModel(
    "claude-sonnet-4-5",
    provider=AnthropicProvider(api_key=ANTHROPIC_API_KEY),
)

## Run Agent

Pydantic AI's [official docs](https://ai.pydantic.dev/agents/)

Agents are Pydantic AI's primary interface for interacting with LLMs.

In [0]:
from pydantic_ai import Agent


agent = Agent(
    model=model,
    # Register static instructions using a keyword argument to the agent.
    # For more complex dynamically-generated instructions, see the example below.
    instructions='Be concise, reply with one sentence.',
)

user_prompt="""Where does "hello world" come from?"""

# Run the agent synchronously, conducting a conversation with the LLM.
result = await agent.run(user_prompt=user_prompt)
print(result.output)

In [0]:
user_prompt = """Hey Claude, explain SOLID principles as if I were five years old."""

result = await agent.run(user_prompt=user_prompt)
print(result.output)

# MLflow Tracking

1. [MLflow Tracking](https://mlflow.org/docs/latest/ml/tracking/)
1. [Token usage tracking](https://docs.databricks.com/aws/en/mlflow3/genai/tracing/integrations/pydantic-ai#token-usage-tracking)

In [0]:
import mlflow

last_trace_id = mlflow.get_last_active_trace_id()
trace = mlflow.get_trace(trace_id=last_trace_id)

print(trace.info.token_usage)
for span in trace.data.spans:
    usage = span.get_attribute("mlflow.chat.tokenUsage")
    if usage:
        print(span.name, usage)

In [0]:
dbutils.notebook.exit("End of demo")

# 👨‍💻 Demo: Bank support

[Bank support](https://ai.pydantic.dev/examples/bank-support/) with some changes:

1. Use Anthropic's `claude-sonnet-4-5` model
1. Use `await agent.run` 

In [0]:
"""Small but complete example of using Pydantic AI to build a support agent for a bank."""

import sqlite3
from dataclasses import dataclass

from pydantic import BaseModel

from pydantic_ai import Agent, RunContext


@dataclass
class DatabaseConn:
    """A wrapper over the SQLite connection."""

    sqlite_conn: sqlite3.Connection

    async def customer_name(self, *, id: int) -> str | None:
        res = cur.execute('SELECT name FROM customers WHERE id=?', (id,))
        row = res.fetchone()
        if row:
            return row[0]
        return None

    async def customer_balance(self, *, id: int) -> float:
        res = cur.execute('SELECT balance FROM customers WHERE id=?', (id,))
        row = res.fetchone()
        if row:
            return row[0]
        else:
            raise ValueError('Customer not found')


@dataclass
class SupportDependencies:
    customer_id: int
    db: DatabaseConn


class SupportOutput(BaseModel):
    support_advice: str
    """Advice returned to the customer"""
    block_card: bool
    """Whether to block their card or not"""
    risk: int
    """Risk level of query"""


support_agent = Agent(
    model=model,
    deps_type=SupportDependencies,
    output_type=SupportOutput,
    instructions=(
        'You are a support agent in our bank, give the '
        'customer support and judge the risk level of their query. '
        "Reply using the customer's name."
    ),
)


@support_agent.instructions
async def add_customer_name(ctx: RunContext[SupportDependencies]) -> str:
    customer_name = await ctx.deps.db.customer_name(id=ctx.deps.customer_id)
    return f"The customer's name is {customer_name!r}"


@support_agent.tool
async def customer_balance(ctx: RunContext[SupportDependencies]) -> str:
    """Returns the customer's current account balance."""
    balance = await ctx.deps.db.customer_balance(
        id=ctx.deps.customer_id,
    )
    return f'${balance:.2f}'

async def main():
    with sqlite3.connect(':memory:') as con:
        cur = con.cursor()
        cur.execute('CREATE TABLE customers(id, name, balance)')
        cur.execute("""
            INSERT INTO customers VALUES
                (123, 'John', 123.45)
        """)
        con.commit()

        deps = SupportDependencies(customer_id=123, db=DatabaseConn(sqlite_conn=con))
        result = await support_agent.run('What is my balance?', deps=deps)
        print(result.output)
        """
        support_advice='Hello John! Your current account balance is $123.45. Is there anything else I can help you with today?' block_card=False risk=1
        """

        result = await support_agent.run('I just lost my card!', deps=deps)
        print(result.output)
        """
        support_advice="Hello John, I understand you've lost your card and I'm here to help you right away. For your security, I'm blocking your card immediately to prevent any unauthorized transactions. You should receive a replacement card within 5-7 business days at your registered address. In the meantime, if you notice any suspicious transactions, please contact us immediately. You can also access your accounts through online banking or our mobile app. Is there anything else I can help you with regarding this matter?" block_card=True risk=8
        """

# Run the agent in a notebook
await main()

# mlflow.pydantic_ai.autolog

## log_traces Argument

Enabled by default

`autolog(log_traces: bool = True)`

Uses `mlflow.start_span` with `SpanType`.
* `span.set_inputs`
* `span.set_outputs`
* `span.set_attribute`

`_patch_method` the following methods:

Module | Class | Method | Span Type
-|-|-|-
`pydantic_ai` | `Agent` | `run` | `SpanType.AGENT`
 &nbsp; | | `run_sync` | `SpanType.AGENT`
 &nbsp; | | `run_stream` | `SpanType.AGENT`
 &nbsp; | | `run_stream_sync` | `SpanType.AGENT`
`pydantic_ai.models.instrumented` | `InstrumentedModel` | `request` | `SpanType.LLM`
 &nbsp; | | `request_stream` | `SpanType.LLM`
`pydantic_ai._tool_manager` | `ToolManager` | `handle_call` | `SpanType.TOOL`
`pydantic_ai.mcp` | `MCPServer` | `call_tool` | `SpanType.TOOL`
 &nbsp; | | `list_tools` | `SpanType.TOOL`
`pydantic_ai` | `Tool` | `run` <br> (if available in the installed version of Pydantic AI) | `SpanType.TOOL`